In [155]:
import sys
sys.path.append('/home/apaulson/repos/data-proc')
import plate_maps as pm

In [156]:
import pandas as pd
import os
import sys
import numpy as np
pd.set_option('display.max_columns', 90)
from pathlib import Path, PureWindowsPath

import importlib
%config Completer.use_jedi=False
print(pd.__version__)

engine = pm.create_mysql_cnx(host='avidddb.ucsf.edu', db='QB3_HiTS_AVIDD')

2.0.3


In [157]:
from sqlalchemy import text

In [158]:
with engine.connect() as conn:
    result=conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


In [159]:
# MetaData stores a collection of tables we want to work with
from sqlalchemy import MetaData
metadata = MetaData()

In [160]:
# reflection gets an existing table from the db
from sqlalchemy import Table

nb=Table("INV_NOTEBOOK", metadata, autoload_with=engine)

# bulk insert new notebooks

In [161]:
query="select * from INV_NOTEBOOK"
nb=pd.read_sql(query, con=engine)
query="select * from DIRECTORY_MEMBER"
dm=pd.read_sql(query, con=engine)

In [162]:
nb.tail(2)

,IN_ID,IN_SDESC,IN_SCIENTIST_ID,IN_ISC_ID,IN_NOTES,IN_STATUS_ID,IN_INS_BY,IN_INS_DATE
50,51,KL196,7,AVIDD,None,1,1,2023-07-31 17:52:28
51,52,Renslo_2GiD287,18,AVIDD,None,1,1,2023-10-16 10:09:12


In [163]:
info=pd.read_csv("/Volumes/Shared/SMDC/All_Compounds/AVIDD/20231016_AVIDD_cpd_reg.csv")
info=info.dropna(how="all", axis="index")
info=info[~info.notebook_name.isna()]
info[['fname','lname']]=info.owner.str.split(' ', expand=True)

In [164]:
dm.DM_ID.nunique(), dm.DM_LNAME.nunique()

(19, 19)

In [165]:
fnmap=dict(zip(dm.DM_LNAME,dm.DM_ID))
info['IN_SCIENTIST_ID']=info.lname.map(fnmap)
info[info.IN_SCIENTIST_ID.isna()]

,project,owner,PARENT_AVIDD,smiles,date_prepared,SCREENING_CENTER,lot_type,vendor_name,lot_alias,notebook_name,notebook_page,appearance,amount_prepared,amount_units,salt,salt_num,solvate,solvate_num,alias1,alias1_value,alias2,alias2_value,alias3,alias3_value,analytical,purity,solubility,stereo,mixture,notes,adduct,fname,lname,IN_SCIENTIST_ID


In [166]:
info.loc[~info.IN_SCIENTIST_ID.isna(), 'IN_SCIENTIST_ID']=info.loc[~info.IN_SCIENTIST_ID.isna(), 'IN_SCIENTIST_ID'].astype(int)

In [167]:
info.notebook_name.unique()

array(['Renslo_1GiD286', 'Renslo_2GiD287', 'QC200', 'ZD193',
       'Deepthi Ramesh Macro'], dtype=object)

In [168]:
from datetime import datetime

In [169]:
info=info[['notebook_name', 'IN_SCIENTIST_ID']].drop_duplicates().dropna(subset='notebook_name')
info=info.rename(columns={'notebook_name':'IN_SDESC'})
info.IN_SDESC=[str(x).replace('.0','') for x in info.IN_SDESC]

info['IN_ISC_ID']='AVIDD'
info['IN_NOTES']=None
info['IN_STATUS_ID']=1
info['IN_INS_BY']=1
info['IN_INS_DATE']=datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [170]:
info.head()

,IN_SDESC,IN_SCIENTIST_ID,IN_ISC_ID,IN_NOTES,IN_STATUS_ID,IN_INS_BY,IN_INS_DATE
0,Renslo_1GiD286,18,AVIDD,None,1,1,2023-10-16 10:30:08
3,Renslo_2GiD287,18,AVIDD,None,1,1,2023-10-16 10:30:08
14,QC200,5,AVIDD,None,1,1,2023-10-16 10:30:08
15,ZD193,6,AVIDD,None,1,1,2023-10-16 10:30:08
37,Deepthi Ramesh Macro,20,AVIDD,None,1,1,2023-10-16 10:30:08


In [171]:
new_entries=(info.IN_SDESC+';'+info.IN_SCIENTIST_ID.astype(str)).tolist()
already_entered=(nb.IN_SDESC+';'+nb.IN_SCIENTIST_ID.astype(str)).tolist()
keep_new=list(set(new_entries)-set(already_entered))
keep_new=[x.split(';')[0] for x in keep_new]
keep_new

['ZD193']

In [172]:
info=info[info.IN_SDESC.isin(keep_new)]
info

,IN_SDESC,IN_SCIENTIST_ID,IN_ISC_ID,IN_NOTES,IN_STATUS_ID,IN_INS_BY,IN_INS_DATE
15,ZD193,6,AVIDD,None,1,1,2023-10-16 10:30:08


In [173]:
# if len(info)>0:
#     info.to_sql(name='INV_NOTEBOOK', con=engine, if_exists = 'append', index=False)

In [174]:
engine.dispose()